In [7]:
import libs.MEVeS as S
import libs.AtomicConstants.Atom as A
import numpy as np
import libs.Optimiser as Opt
import pickle

In [5]:
### Make atom class
config = {"Hyperfine splitting": False, "states": {"initial": {"F": 4}, "intermediate": {"n": 6, "J": 3/2}, 
                                                    "storage": {"n": 6, "L":0, "F" : 3}}}
atom = A.Cs(config)

T = 0 # temperature of the ensemble in K
vno = 1 # number of velocity classes to simulate.
L = 0.075 # length of the cell in m

OD = 5000

n = 50 # number of space steps. Note these are not evenly spaced.

Sinits = np.zeros((n, len(atom.Fg), len(atom.mg), len(atom.Fq), len(atom.mq), vno)) # initial spin wave

### Make solver class
protocol = 'Raman'
tau = 1e-9
m = 800
detuning = 2*np.pi*2*atom.deltaHF

deltaS = detuning #definition of detuning means that signal field and control field should be negative of each other
                    #for two photon resonance
deltaC = detuning

t0 = 3*tau
tend = 4*t0
tbounds = np.array([0, tend])

t = np.linspace(0, tend, m) #this is also defined in solver but in natural units -> tidy up?

photon_pol = np.array([1, 0]) #should be normalised
Einits = np.array(S.photon_gaussian(t, t0, tau))[:, None] * photon_pol[None, :] 

control_pol = np.array([1, 0]) #should be normalised

sol = S.solver(atom, protocol, Einits, Sinits, [deltaS, deltaC], OD, L, n, m, tbounds, T, vno)

In [6]:
### Make optimiser class
Control_init1 = S.gaussian(t, t0, tau, 1e10)[:, None] * control_pol[None, :]
Control_init2 = S.gaussian(t, t0, tau, 1e10)[:, None] * control_pol[None, :]
Control_inits = [Control_init1, Control_init2]
step_size = 300

opt = Opt.optimiser(sol, method='RK4-Ralston', _complex=True, verbose=False, live_plot=True, save_to_file=True)
opt.max_points = 10
tol = 1e-6
opt.beta1 = 0.9

eff, Control_opts, count = opt.Krotov_momentum(opt.forward_retrieval_opt_all, Control_inits, step_size, tol,
                                    adaptive_step_size=True)
print(f"OD={OD}, total_eff={eff}\n")
print(f"Max amps: readin={np.max(np.abs(Control_opts[0][:, 0]))}, readout={np.max(np.abs(Control_opts[1][:, 0]))}\n")
print(f"Total energy: readin={atom.control_pulse_to_energy(np.abs(Control_opts[0][:, 0]), t, r=250e-6, index=1)}, readout={atom.control_pulse_to_energy(np.abs(Control_opts[1][:, 0]), t, r=250e-6, index=1)}\n")

    'data': [{'mode': 'lines+markers',
              'name': 'Storage Efficiency…

OD=5000, total_eff=0.49143865360015593

Max amps: readin=17931159537.23722, readout=22413174244.471127

Total energy: readin=4.084309064652902e-09, readout=6.078605439157485e-09



In [9]:
# Unpickling the object
with open('2024-06-16_12-29-58.pkl', 'rb') as file:
    loaded_sol = pickle.load(file)

In [19]:
loaded_sol.atom

Cs_6s1q2F[0]_6p3q2_6s1q2F[0]_HyperfineSplittingNo

In [15]:
with np.load('2024-06-16_12-29-58.npz') as data:
    Controls_opt = data['Control_best']
    Controls_inits = data['Control_init']

In [16]:
Control_tzp = loaded_sol.co_prop( Controls_opt[0] )
loaded_sol.solve(Control_tzp, method='RK4-Ralston')
storage = loaded_sol.storage_efficiency(loaded_sol.S, -1)
print(f"Storage efficiency = {storage*100: .2f} %")

Storage efficiency =  71.14 %
